In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')

X_train = train.iloc[:,1:-1]
y_train = train.iloc[:,-1]
X_test = test.iloc[:,1:]
X_train.shape, y_train.shape, X_test.shape

In [ ]:
from sklearn.metrics import make_scorer

def loss(y_true,y_pred):
    # gt is short for ground -truth
    # it is a matrix in which each line has only one 1 and others are 0
    # you can get it from get_dummies(df.target)
    # 
    # pred is from clf.pred_proba()
    
    assert y_pred.shape == y_true.shape
    
    sum = 0
    for i in y_pred * y_true:
        sum += np.log(
            min( max(10e-15, np.sum(i)), 1 - 10e-15)
        )
    return sum * (-1) * (1/y_pred.shape[0])

def my_scorer(y_true, y_pred_proba):
    y_true_mat = pd.get_dummies(y_true).values
    error = loss(y_true_mat, y_pred_proba)
    return error

the_scorer = make_scorer(my_scorer, needs_proba=True, greater_is_better=False)

clf.get_params()
```
{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}
 ```

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

# grid = GridSearchCV(
#     estimator=lgb.LGBMClassifier(),
#     param_grid={
#         'n_estimators':[800],
#         'learning_rate':[0.01]
#     },
#     scoring=the_scorer,
#     verbose=3
# )
# grid.fit(X_train, y_train)

In [ ]:
# for result in grid.cv_results_:
#     print(result, grid.cv_results_[result])
# grid.best_params_

In [ ]:
clf = lgb.LGBMClassifier(n_estimators=800, learning_rate=0.01)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
y_pred

In [ ]:
# clf = lgb.LGBMClassifier()
# clf.fit(X_train, y_train)

# y_pred = clf.predict_proba(X_test)
# y_pred

In [ ]:
submit = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
submit

In [ ]:
my_submit = pd.DataFrame()
# my_submit = pd.DataFrame(y_pred,columns=submit.columns[1:])
my_submit['id'] = submit.id
my_submit[submit.columns[1:]] = y_pred
my_submit


In [ ]:
my_submit.to_csv('my_submit_lgbm-02.csv',index=False)
reload = pd.read_csv('my_submit_lgbm-02.csv')
reload
